### Initialize

In [2]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='CHANGEME')

### Generate TIF

In [72]:
# Function to process a single point
def process_point(lon, lat, city, year, filename):
    # Define the center point
    center = ee.Geometry.Point([lon, lat])

    # Define the bounding box with a buffer radius (2560 meters for 512x512 pixels at 10m resolution)
    geometry = center.buffer(2560).bounds()

    # Load Sentinel-2 MSI Level-2A as ImageCollection
    sentinel2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                 .filterBounds(geometry)
                 .filterDate(f'{year}-08-01', f'{year}-08-30')
                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
                 .median())

    # Calculate NDVI
    ndvi = sentinel2.normalizedDifference(['B8', 'B4']).rename('NDVI')

    # Classify NDVI into three categories
    ndvi_classified = ndvi.expression(
        "(ndvi < 0.4) ? 1 : (ndvi <= 0.7) ? 2 : 3",
        {'ndvi': ndvi}
    ).rename('NDVI_Classified')

    # Visualization parameters for the classified NDVI
    ndvi_class_vis = {
        'min': 1,
        'max': 3,
        'palette': ['white', 'lightgreen', 'darkgreen']  # Assign colors to each class
    }

    # Enhance each band by a different factor
    enhanced_rgb = (sentinel2.select(['B4', 'B3', 'B2']).multiply(2.0))

    viz_params = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 3000,
        'gamma': 1
    }

    # Export EnhancedRGB Layer to Google Drive
    enhanced_rgb_task = ee.batch.Export.image.toDrive(
        image=enhanced_rgb.clip(geometry).visualize(**viz_params),
        description=f'EnhancedRGB_{filename}',
        folder=year,
        fileNamePrefix=filename,
        region=geometry,
        fileFormat='GEOTIFF',
        crs='EPSG:3857',
        dimensions='512x512'
    )
    enhanced_rgb_task.start()

    # Export the classified NDVI as a single-band image
    ndvi_classified_task = ee.batch.Export.image.toDrive(
        image=ndvi_classified.clip(geometry),
        description=f'NDVI_Classified_{filename}',
        folder=year,
        fileNamePrefix=f'm_{filename}',
        region=geometry,
        fileFormat='GEOTIFF',
        crs='EPSG:3857',
        dimensions='512x512'
    )
    ndvi_classified_task.start()

    print(f"Tasks started for {city} at ({lon}, {lat}).")


### Data

In [73]:
import io
import pandas as pd
data = pd.read_csv(io.StringIO('''
Sacramento,2019,2024,"-121.387740466012,38.6049281216723",38.60492812,-121.3877405,"-121.387740466012,38.6549281216723","-121.337740466012,38.6549281216723","-121.337740466012,38.6049281216723","-121.337740466012,38.5549281216723","-121.387740466012,38.5549281216723","-121.437740466012,38.5549281216723","-121.437740466012,38.6049281216723","-121.437740466012,38.6549281216723"
SanFrancisco,2019,2024,"-121.76414419757,37.8162582620558",37.81625826,-121.7641442,"-121.76414419757,37.8662582620558","-121.71414419757,37.8662582620558","-121.71414419757,37.8162582620558","-121.71414419757,37.7662582620558","-121.76414419757,37.7662582620558","-121.81414419757,37.7662582620558","-121.81414419757,37.8162582620558","-121.81414419757,37.8662582620558"
Fresno,2019,2024,"-119.417894258498,36.7078861066282",36.70788611,-119.4178943,"-119.417894258498,36.7578861066282","-119.367894258498,36.7578861066282","-119.367894258498,36.7078861066282","-119.367894258498,36.6578861066282","-119.417894258498,36.6578861066282","-119.467894258498,36.6578861066282","-119.467894258498,36.7078861066282","-119.467894258498,36.7578861066282"
Bakersfield,2019,2024,"-118.439244556448,35.3789556757796",35.37895568,-118.4392446,"-118.439244556448,35.4289556757796","-118.389244556448,35.4289556757796","-118.389244556448,35.3789556757796","-118.389244556448,35.3289556757796","-118.439244556448,35.3289556757796","-118.489244556448,35.3289556757796","-118.489244556448,35.3789556757796","-118.489244556448,35.4289556757796"
'''), header=None)


### RUN

In [75]:
coord_list = [3, 6, 7, 8, 9, 10, 11, 12, 13]
coord_name = ['C', 'N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']

# Process each city
for j in range(3):
  cityData = data.iloc[j]
  city = cityData[0]
  print(city)
  for i in range(9):
      coord = cityData.iloc[coord_list[i]].split(',')
      filename = f'{city}_{coord_name[i]}'
      # print(filename)

      lon = float(coord[0])
      lat = float(coord[1])
      # print(lon, lat)
      process_point(lon, lat, city, '2019', filename)
      process_point(lon, lat, city, '2024', filename)


print("All tasks have been started.")

Sacramento
Tasks started for Sacramento at (-121.387740466012, 38.6049281216723).
Tasks started for Sacramento at (-121.387740466012, 38.6049281216723).
Tasks started for Sacramento at (-121.387740466012, 38.6549281216723).
Tasks started for Sacramento at (-121.387740466012, 38.6549281216723).
Tasks started for Sacramento at (-121.337740466012, 38.6549281216723).
Tasks started for Sacramento at (-121.337740466012, 38.6549281216723).
Tasks started for Sacramento at (-121.337740466012, 38.6049281216723).
Tasks started for Sacramento at (-121.337740466012, 38.6049281216723).
Tasks started for Sacramento at (-121.337740466012, 38.5549281216723).
Tasks started for Sacramento at (-121.337740466012, 38.5549281216723).
Tasks started for Sacramento at (-121.387740466012, 38.5549281216723).
Tasks started for Sacramento at (-121.387740466012, 38.5549281216723).
Tasks started for Sacramento at (-121.437740466012, 38.5549281216723).
Tasks started for Sacramento at (-121.437740466012, 38.55492812167

### The Code Not Taken

#### Single City

In [68]:
import io
import pandas as pd
data = pd.read_csv(io.StringIO('''
SanJose,2019,2024,"-121.899180305122,37.3391025913052",37.33910259,-121.8991803,"-121.899180305122,37.3891025913052","-121.849180305122,37.3891025913052","-121.849180305122,37.3391025913052","-121.849180305122,37.2891025913052","-121.899180305122,37.2891025913052","-121.949180305122,37.2891025913052","-121.949180305122,37.3391025913052","-121.949180305122,37.3891025913052"
'''), header=None)


In [71]:
coord_list = [3, 6, 7, 8, 9, 10, 11, 12, 13]
coord_name = ['C', 'N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
city='SanJose'
# Process each point
for i in range(9):
    coord = data.iloc[0, coord_list[i]].split(',')
    filename = f'{city}_{coord_name[i]}'
    lon = float(coord[0])
    lat = float(coord[1])

    print(filename)
    print(lon, lat)
    process_point(lon, lat, city, '2019', filename)

print("All tasks have been started.")


SanJose_C
-121.899180305122 37.3391025913052
Tasks started for SanJose at (-121.899180305122, 37.3391025913052).
SanJose_N
-121.899180305122 37.3891025913052
Tasks started for SanJose at (-121.899180305122, 37.3891025913052).
SanJose_NE
-121.849180305122 37.3891025913052
Tasks started for SanJose at (-121.849180305122, 37.3891025913052).
SanJose_E
-121.849180305122 37.3391025913052
Tasks started for SanJose at (-121.849180305122, 37.3391025913052).
SanJose_SE
-121.849180305122 37.2891025913052
Tasks started for SanJose at (-121.849180305122, 37.2891025913052).
SanJose_S
-121.899180305122 37.2891025913052
Tasks started for SanJose at (-121.899180305122, 37.2891025913052).
SanJose_SW
-121.949180305122 37.2891025913052
Tasks started for SanJose at (-121.949180305122, 37.2891025913052).
SanJose_W
-121.949180305122 37.3391025913052
Tasks started for SanJose at (-121.949180305122, 37.3391025913052).
SanJose_NW
-121.949180305122 37.3891025913052
Tasks started for SanJose at (-121.949180305122